# Comparing diferent machine learning techniques to predict football results
# - Authors and affiliations
- Abstract
- 1. Introduction

# 1. Introduction

As one of the most popular sports on the planet, football has always been followed
very closely by a large number of people. In recent years, new types of data have
been collected for many games in various countries, such as play-by-play data including information on each shot or pass made in a match.

Information like this can be found in datasets like [the Football Events dataset](https://datasetsearch.research.google.com/search?docid=beC2NjeMuiLj9GvLAAAAAA%3D%3D) found on kaggle.

This projects objective is to use the above mentioned dataset to predict the number of goals on a football game using Regression algorithms.
This paper will start with an explanation and analysis of the used dataset, followed by an explanation of the approach the was decided to be used.

Tests will be performed on the developed models and explanation for the results will be given at the end.

# 2. Description of the problem/dataset

Our data set has two files of data and one dictionary. The first file (events.csv) gives information on all the
recorded events with 941,009 events for 9,074 games. The second file (ginf.csv) gives the details of the
odds for the games recorded in the first file . for each league we have information on the seasons
from 2012 to 2017 except the English league which information only starts in the season 2014.
The dictionary helps us to understands the values in some of the columns of the events table.

We will focus more on the first table which will be the most useful since we not using odds.

## 2.1. events.csv Columns

1. id odsp : Unique identifier of the game (odsp stands from oddsportal.com)
2. id event : Unique identifier of event (id odsp + sort order)
3. sort order : Chronological sequence of events in a game
4. time : Minute of the game when the event happened
5. text : Text commentary
6. event type : Primary event. 11 unique events (1-Attempt(shot), 2-Corner, 3-Foul, 4-Yellow Card, 5-Second Yellow Card, 6-(Straight) Red Card, 7-Substitution, 8-Free Kick Won, 9-Offside, 10-Hand Ball, 11-Penalty conceded)
7. event type2 : Cecondary event. 4 unique events (12 - Key Pass, 13 - Failed through ball, 14-Sending off, 15-Own goal)
8. side : 1-Home, 2-Away
9. event team : Team that produced the event. In case of Own goals, event team is the team that benefited from the own goal
10. opponent : Team that the event happened against
11. player : Name of the player involved in main event (converted to lowercase and special
chars were removed)
12. player2 : Name of player involved in secondary event 8
13. player in : Player that came in (only applies to substitutions)
14. player out : Player substituted (only applies to substitutions)
15. shot place : Placement of the shot (13 possible placement locations, available in the
dictionary, only applies to shots)
16. shot outcome : 4 possible outcomes (1-On target, 2-Off target, 3-Blocked, 4-Hit the post)
17. is goal : Binary variable if the shot resulted in a goal (own goals included)
18. location : Location on the pitch where the event happened (19 possible locations, available
in the dictionary)
19. bodypart : (1- right foot, 2-left foot, 3-head)
20. assist method : In case of an assisted shot, 5 possible assist methods (details in the dictionary)
21. situation : 4 types: 1-Open Play, 2-Set piece (excluding Direct Free kicks), 3-Corner, 4-Free kick

## 2.2 Loading the Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib import pyplot
import scipy as sp
import seaborn as sb

From the website we also know that the missing values are set as 'NA'so we should treat them as such

In [2]:
events = pd.read_csv('football-events/events.csv',na_values=['NA'])
games = pd.read_csv('football-events/ginf.csv',na_values=['NA'])

In [3]:
events.describe()

,sort_order,time,event_type,event_type2,side,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
count,941009.000000,941009.000000,941009.000000,214293.000000,941009.000000,227459.000000,228498.000000,941009.000000,467067.000000,229185.000000,941009.000000,229137.000000,941009.000000
mean,53.858826,49.663663,4.326575,12.233764,1.481170,5.733693,1.926555,0.025978,6.209073,1.624831,0.264332,1.281316,0.004876
std,32.014268,26.488977,2.995313,0.468850,0.499646,3.326100,0.797055,0.159071,5.421736,0.740400,0.655501,0.709394,0.069655
min,1.000000,0.000000,1.000000,12.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000
25%,27.000000,27.000000,2.000000,12.000000,1.000000,2.000000,1.000000,0.000000,2.000000,1.000000,0.000000,1.000000,0.000000
50%,53.000000,51.000000,3.000000,12.000000,1.000000,5.000000,2.000000,0.000000,3.000000,1.000000,0.000000,1.000000,0.000000
75%,79.000000,73.000000,8.000000,12.000000,2.000000,9.000000,3.000000,0.000000,11.000000,2.000000,0.000000,1.000000,0.000000
max,180.000000,100.000000,11.000000,15.000000,2.000000,13.000000,4.000000,1.000000,19.000000,3.000000,4.000000,4.000000,1.000000


We might be dealing with events that dont have a corresponding game in the ```ginf.csv``` table. So let's verify that first.

We can check if there are any 'rogue events' by joining both tables on the game id and checking if there is any loss of rows.  

In [4]:
unique_games = games['id_odsp'].nunique()
unique_documented_games = events['id_odsp'].nunique()
event_number = events['id_odsp'].size
print(f'Number of events = {event_number}')
merged_size = pd.merge(events, games, right_on='id_odsp', left_on='id_odsp', how='inner')['id_odsp'].size
print(f'Number of events after merge = {merged_size}')

Number of events = 941009
Number of events after merge = 941009


Since there were no lost events after the merge, we can rest assured that there are no rogue events in this dataset.

The dataset's website also states that only around 90% of the games in ```ginf.csv``` have events in ```events.csv```.

In [11]:
print(f'{unique_games} unique games.')
print(f'{unique_documented_games} unique documented games.')
print(f'{unique_games - unique_documented_games} unique undocumented games.')
print(f'{unique_documented_games/unique_games * 100:.2f}% of games are documented.')

10112 unique games.
9074 unique documented games.
1038 unique undocumented games.
89.73% of games are documented.


In [7]:
## 3. Approach

## 5. Conclusions

The data set, and inherently the model, are flawed.
In essence, football games present an extremelly non deterministic world. 
The number of goals a team scores doesn't depend only on the recorded statistics, it also depends on player health, team state of mind, factors that are foreign to the games like club internall conflicts, etc.

This leads to situations were two games of a team that have exactly the same statistics will have different results, due to the circumstances of that time. 
It also makes it hard to find outliers as we can't choose which behaviour is normal.
All of this added together makes regression models have a hard time being accurate.

Another problem is that the dataset holds is missing some data that would be extremelly useful, like ball possession percentages.